In [134]:
import numpy as np
from torch.nn import ConvTranspose2d
from torch import from_numpy
from numpy.testing import assert_array_equal

In [135]:
def conv2d_extra(input, weight, bias, in_channels, out_channels, kernel_size, stride=1, padding=0,
                            dilation=1, bias_enabled=True, padding_mode='zeros'):
    batches = len(input)
    out = []

    for b in range(batches):
        h_in, w_in = input[b].shape[1], input[b].shape[2]

        kernel_size = (kernel_size, kernel_size) if not isinstance(kernel_size, tuple) else kernel_size
        stride = (stride, stride) if not isinstance(stride, tuple) else stride
        dilation = (dilation, dilation) if not isinstance(dilation, tuple) else dilation

        padding = padding if isinstance(padding, tuple) else (padding, padding)

        strided = stride_input(input[b], stride)
        h_in, w_in = strided.shape[1], strided.shape[2]
        
        h_out = int((h_in - 1) - 2 * padding[0] + dilation[0] * (kernel_size[0] - 1) + 1)
        w_out = int((w_in - 1) - 2 * padding[1] + dilation[1] * (kernel_size[1] - 1) + 1)
        
        out.append([[[0. for _ in range(w_out)] for _ in range(h_out)] for _ in range(out_channels)])
        for c_out in range(out_channels):
            for y_out in range(h_out):
                for x_out in range(w_out):
                    summation = 0
                    for c_in in range(in_channels):
                        for kernel_y in range(kernel_size[0]):
                            for kernel_x in range(kernel_size[1]):
                                y_in = y_out + kernel_y * dilation[0] - (dilation[0] * (kernel_size[0] - 1) - padding[0])
                                x_in = x_out + kernel_x * dilation[1] - (dilation[1] * (kernel_size[1] - 1) - padding[1])
                                if 0 <= y_in < h_in and 0 <= x_in < w_in:
                                    summation += strided[c_in][y_in][x_in] * weight[c_out][c_in][kernel_y][kernel_x]

                    out[b][c_out][y_out][x_out] = summation + (bias[c_out] if bias_enabled else 0)

    return np.array(out)

In [136]:
def stride_input(input, stride):
    channels, rows, cols = input.shape
    out_rows, out_cols = rows * stride[0] - stride[0] + 1, cols * stride[1] - stride[1] + 1
    out = np.zeros((channels, out_rows, out_cols), input.dtype)

    for c in range(channels):
        for y_out in range(0, out_rows, stride[0]):
            for x_out in range(0, out_cols, stride[1]):
                y = int(y_out / stride[0])
                x = int(x_out / stride[1])
                out[c, y_out, x_out] = input[c, y, x]

    return out

In [137]:
def deffault_data(bias = None):
    channel_1 = [[1, 5, 5, 6], [7, 11, 2, 11], [4, 8, 5, 3],[1, 2, 3, 4]]
    channel_2 = [[1, 5, 5, 6], [7, 11, 2, 11], [4, 8, 5, 3],[1, 2, 3, 4]]

    weight_1 = [[1, 1],[1, 1]]
    weight_2 = [[1, 1],[1, 1]]

    input = np.array([[channel_1, channel_2]])
    weight = np.array([[weight_1, weight_2], [weight_1, weight_2]])
    bias = np.array([0, 0] if bias is None else bias)

    return input, weight, bias

In [138]:
def assert_equal_own_and_torch(own, torch, input, weight, bias):
    torch.weight.data = from_numpy(weight).float()
    torch.bias.data = from_numpy(bias).float()
    own_result = np.floor(own)
    torch_result = np.floor(torch(from_numpy(input).float()).detach().numpy())
    #print(np.array(own_result).shape)
    #print(np.array(torch_result).shape)
    print("Мой результат")
    print(own_result)
    print("Результат библиотеки")
    print(torch_result)
    assert_array_equal(own_result, torch_result)

In [139]:
def test():
    input, weight, bias = deffault_data()

    print("Stride test:")
    own_result = conv2d_extra(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, stride=2)
    torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, stride=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Kernel size test:")
    own_result = conv2d_extra(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2)
    torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Padding test:")
    own_result = conv2d_extra(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=1)
    torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, padding=1)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Padding test:")
    own_result = conv2d_extra(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=2)
    torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, padding=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Dilation test:")
    own_result = conv2d_extra(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, dilation=2)
    torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, dilation=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Bias test:")
    own_result = conv2d_extra(input, weight, bias=[1, 1], in_channels=2, out_channels=2, kernel_size=2)
    torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, bias=True)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias=np.array([1,1]))

    print("Padding mode test:")
    own_result = conv2d_extra(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=1, padding_mode='zeros')
    torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, padding=1, padding_mode='zeros')
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

In [140]:
test()

Stride test:
Мой результат
[[[[ 2.  2. 10. 10. 10. 10. 12. 12.]
   [ 2.  2. 10. 10. 10. 10. 12. 12.]
   [14. 14. 22. 22.  4.  4. 22. 22.]
   [14. 14. 22. 22.  4.  4. 22. 22.]
   [ 8.  8. 16. 16. 10. 10.  6.  6.]
   [ 8.  8. 16. 16. 10. 10.  6.  6.]
   [ 2.  2.  4.  4.  6.  6.  8.  8.]
   [ 2.  2.  4.  4.  6.  6.  8.  8.]]

  [[ 2.  2. 10. 10. 10. 10. 12. 12.]
   [ 2.  2. 10. 10. 10. 10. 12. 12.]
   [14. 14. 22. 22.  4.  4. 22. 22.]
   [14. 14. 22. 22.  4.  4. 22. 22.]
   [ 8.  8. 16. 16. 10. 10.  6.  6.]
   [ 8.  8. 16. 16. 10. 10.  6.  6.]
   [ 2.  2.  4.  4.  6.  6.  8.  8.]
   [ 2.  2.  4.  4.  6.  6.  8.  8.]]]]
Результат библиотеки
[[[[ 2.  2. 10. 10. 10. 10. 12. 12.]
   [ 2.  2. 10. 10. 10. 10. 12. 12.]
   [14. 14. 22. 22.  4.  4. 22. 22.]
   [14. 14. 22. 22.  4.  4. 22. 22.]
   [ 8.  8. 16. 16. 10. 10.  6.  6.]
   [ 8.  8. 16. 16. 10. 10.  6.  6.]
   [ 2.  2.  4.  4.  6.  6.  8.  8.]
   [ 2.  2.  4.  4.  6.  6.  8.  8.]]

  [[ 2.  2. 10. 10. 10. 10. 12. 12.]
   [ 2.  2. 10. 10. 